In [1]:
# %load_ext autoreload
# %autoreload
# %matplotlib inline

from news_crawler import NewsCrawler
import glob
import json
from IPython.core.debugger import set_trace
from pathlib import Path
from pprint import pprint
import spacy
from tokenizer import MergedTokenizer
import time

import gensim
import gensim.corpora as corpora

import pyLDAvis
import pyLDAvis.gensim as gensimvis

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

from tqdm.auto import tqdm
tqdm.pandas()

C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
nlp = spacy.load('en_core_web_lg')

In [4]:
fnames = glob.glob('newsdata/downloaded/*.json')[:50]

In [4]:
merged_tokenizer = MergedTokenizer(nlp, merge_chunks=False, merge_ner=True)

In [5]:
data = []
for fname in tqdm(fnames):
    js = json.loads(Path(fname).read_text())
    doc = merged_tokenizer.lemmatized(js['text'])
    data.append(doc)

In [6]:
id2word = corpora.Dictionary(data)

In [7]:
corpus = [id2word.doc2bow(text) for text in data]

In [8]:
s = time.time()
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                            id2word=id2word, 
                                            num_topics=5, 
                                            random_state=100, 
                                            update_every=None, 
                                            chunksize=1, 
                                            passes=5, 
                                            alpha='auto', 
                                            per_word_topics=True)
pprint(time.time()-s)

8.586445569992065


In [11]:
lda_model = gensim.models.HdpModel(corpus=corpus, id2word=id2word)

In [12]:
lda_model.show_topics()

[(0,
  '0.012*say + 0.004*new + 0.004*time + 0.004*people + 0.003*trump + 0.003*president + 0.002*state + 0.002*company + 0.002*getty + 0.002*world + 0.002*come + 0.002*image + 0.002*tell + 0.002*go + 0.002*work + 0.002*good + 0.002*know + 0.002*report + 0.002*include + 0.002*high'),
 (1,
  '0.012*say + 0.004*trump + 0.004*time + 0.004*new + 0.003*people + 0.003*president + 0.003*good + 0.003*state + 0.003*go + 0.003*tell + 0.003*work + 0.002*come + 0.002*know + 0.002*want + 0.002*take + 0.002*house + 0.002*report + 0.002*think + 0.002*company + 0.002*world'),
 (2,
  '0.009*say + 0.003*time + 0.002*people + 0.002*new + 0.002*company + 0.002*work + 0.002*know + 0.002*go + 0.002*good + 0.002*start + 0.002*outstanding + 0.002*state + 0.001*game + 0.001*product + 0.001*come + 0.001*change + 0.001*shampoo + 0.001*tell + 0.001*want + 0.001*add'),
 (3,
  '0.007*say + 0.006*trump + 0.003*people + 0.003*president + 0.002*getty + 0.002*time + 0.002*new + 0.002*think + 0.002*chinese + 0.002*come 

In [17]:
pyLDAvis.enable_notebook()

In [ ]:
'''
여기를 실행시키려면 반드시 future 패키지를 0.18로 업그레이드한다
pip install future --upgrade
'''

vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [2]:
import glob
import json
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
# warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint
from smart_open import smart_open
from pathlib import Path

from tqdm.auto import tqdm
tqdm.pandas()

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\infomax\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\infomax\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [38]:
def build_texts(date):
    fnames = glob.glob('newsdata/downloaded/*.json')
    for fname in tqdm(fnames[:30000]):
        js = json.loads(Path(fname).read_text())
        if js['published_at']==date:
            yield gensim.utils.simple_preprocess(js['text'], deacc=True, min_len=3)

In [50]:
train_texts = list(build_texts('2019-10-13'))

In [51]:
len(train_texts)

666

In [52]:
bigram = gensim.models.Phrases(train_texts)

In [53]:
from gensim.utils import lemmatize
from nltk.corpus import stopwords

In [54]:
stops = set(stopwords.words('english'))

In [55]:
def process_texts(texts):
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    #texts = [[word for word in lemmatizer.lemmatize(' '.join(line), pos='v').split()] for line in texts]
    texts = [[lemmatizer.lemmatize(word, pos='v') for word in line] for line in texts]
    return texts

In [56]:
texts = process_texts(train_texts)

In [57]:
texts[0]

['surprise',
 'one',
 'donald_trump',
 'factor',
 'heavily',
 'hate',
 'six',
 'part',
 'discovery',
 'channel',
 'documentary',
 'series',
 'executive',
 'produce',
 'steven',
 'spielberg',
 'alex',
 'gibney',
 'investigate',
 'many',
 'form',
 'cause',
 'possible',
 'remedy',
 'intolerance',
 'ubiquitous',
 'presence',
 'fact',
 'even',
 'specific',
 'topic',
 'conversation',
 'corrosive',
 'shadow',
 'loom',
 'large',
 'proceed',
 'international',
 'criminal',
 'lawyer',
 'patricia',
 'viseur',
 'sellers',
 'describe',
 'khmer',
 'rouge',
 'madman',
 'pol',
 'pot',
 'motivations',
 'initiate',
 'genocide',
 'state',
 'pause',
 'indicate',
 'know',
 'imply',
 'go',
 'make',
 'cambodia',
 'great',
 'current',
 'president',
 'ugly',
 'divisive',
 'language',
 'policies',
 'white',
 'nationalist',
 'movements',
 'mass',
 'shoot',
 'inspire',
 'natural',
 'fit',
 'directors',
 'geeta',
 'gandbhir',
 'sam',
 'pollard',
 'non',
 'fiction',
 'work',
 'premier',
 'oct',
 'trump',
 'perfectly

In [58]:
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [59]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [36]:
lsimodel.show_topics(num_topics=5, formatted=False)

[(0,
  [('noon', -0.6613216968723826),
   ('lane', -0.46454260670204767),
   ('ave', -0.2841907711589004),
   ('way', -0.2583361514239686),
   ('lane_noon', -0.13663017844177874),
   ('hill', -0.1096466067462519),
   ('ave_noon', -0.09837372847808079),
   ('way_noon', -0.09837372847808079),
   ('park', -0.09349149630332552),
   ('county', -0.08767795208114522)]),
 (1,
  [('say', 0.49071339033150546),
   ('one', 0.16044114629089629),
   ('also', 0.1437593795008963),
   ('jesse', 0.1379700929949128),
   ('make', 0.13474691104042738),
   ('get', 0.11741075749882347),
   ('would', 0.11709819554865286),
   ('china', 0.11259388004910137),
   ('take', 0.10266960571547686),
   ('people', 0.10163539160551438)]),
 (2,
  [('jesse', -0.680802036078172),
   ('walt', -0.38078757950135045),
   ('say', 0.24994723380790815),
   ('gus', -0.11539017560646977),
   ('get', -0.10549742394335948),
   ('meth', -0.10385115804582276),
   ('breaking_bad', -0.09231214048517587),
   ('drug', -0.09191937442831924),

In [60]:
hdpmodel = HdpModel(corpus=corpus, id2word=dictionary, T=20)

In [63]:
hdpmodel.show_topics(formatted=False)

[(0,
  [('say', 0.005884541485610491),
   ('one', 0.002371942988922236),
   ('make', 0.0021407750764234085),
   ('get', 0.0019982344202957294),
   ('would', 0.0019463104649149335),
   ('time', 0.0017699762620776362),
   ('also', 0.0017368933982752149),
   ('trump', 0.0015903226847875237),
   ('first', 0.001554274071399876),
   ('take', 0.0015145843487839998),
   ('tell', 0.0014255771972974127),
   ('call', 0.0013687144621535577),
   ('leave', 0.0013238399434951484),
   ('go', 0.001306015342142846),
   ('come', 0.0012997663406222896),
   ('two', 0.0012916336424684597),
   ('people', 0.001217814198073426),
   ('think', 0.0011761841863738567),
   ('could', 0.0011236903773413844),
   ('show', 0.0011149810754392977)]),
 (1,
  [('say', 0.002402569523054838),
   ('one', 0.0011373089779834708),
   ('make', 0.0010735433574420186),
   ('police', 0.0009885968127852232),
   ('bank', 0.0009214234531433054),
   ('back', 0.0008759720156083504),
   ('get', 0.0008469150020592062),
   ('time', 0.0008226

In [64]:
ldamodel = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary, per_word_topics=True)

In [67]:
ldamodel.show_topics(formatted=False, num_topics=-1)

[(0,
  [('say', 0.012105019),
   ('trump', 0.0044929725),
   ('one', 0.0040703225),
   ('make', 0.003906528),
   ('take', 0.0036466315),
   ('get', 0.0034207036),
   ('time', 0.0034133247),
   ('also', 0.0030831806),
   ('come', 0.0029170031),
   ('would', 0.0028666982)]),
 (1,
  [('say', 0.008835403),
   ('make', 0.0043175006),
   ('one', 0.0036365548),
   ('also', 0.0035651387),
   ('would', 0.0033202178),
   ('get', 0.003169171),
   ('people', 0.003114478),
   ('take', 0.0031086125),
   ('time', 0.00257321),
   ('back', 0.0022962068)]),
 (2,
  [('say', 0.010314101),
   ('like', 0.0040597846),
   ('would', 0.0031793474),
   ('make', 0.0031490894),
   ('also', 0.0030099703),
   ('people', 0.0029942642),
   ('trump', 0.0029528618),
   ('show', 0.0027950713),
   ('one', 0.0027707345),
   ('time', 0.0026653768)]),
 (3,
  [('say', 0.020314895),
   ('one', 0.0040646484),
   ('make', 0.003924041),
   ('take', 0.003417893),
   ('would', 0.0033503133),
   ('get', 0.0032626297),
   ('sunday', 

In [74]:
def ret_top_model():
    """
    Since LDAmodel is a probabilistic model, it comes up different topics each time we run it. To control the
    quality of the topic model we produce, we can see what the interpretability of the best topic is and keep
    evaluating the topic model until this threshold is crossed. 
    
    Returns:
    -------
    lm: Final evaluated topic model
    top_topics: ranked topics in decreasing order. List of tuples
    """
    top_topics = [(0, 0)]
    while top_topics[0][1] < 0.5:
        lm = LdaModel(corpus=corpus, id2word=dictionary)
        coherence_values = {}
        for n, topic in lm.show_topics(num_topics=-1, formatted=False):
            topic = [word for word, _ in topic]
            cm = CoherenceModel(topics=[topic], texts=texts, dictionary=dictionary, window_size=10)
            coherence_values[n] = cm.get_coherence()
        top_topics = sorted(coherence_values.items(), key=operator.itemgetter(1), reverse=True)
    return lm, top_topics

In [ ]:
lm, top_topics = ret_top_model()

In [29]:
import pyLDAvis.gensim

In [30]:
pyLDAvis.enable_notebook()

In [37]:
# %%prun
vis = pyLDAvis.gensim.prepare(lsimodel, corpus, dictionary)

AttributeError: 'LsiModel' object has no attribute 'inference'

In [33]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.033061  0.012004       1        1  23.022216
1      0.020689  0.003203       2        1  12.261437
2      0.029141  0.029430       3        1  12.084545
3      0.040560 -0.004921       4        1  10.282943
5     -0.006983  0.003329       5        1   8.754096
6      0.013125 -0.002872       6        1   8.174843
7      0.011992  0.015940       7        1   4.394865
8      0.026280 -0.031351       8        1   3.845271
10    -0.017314  0.008402       9        1   3.666328
9      0.036619  0.043452      10        1   2.771118
4      0.040660 -0.005328      11        1   2.548378
12     0.012090  0.001472      12        1   2.487806
18    -0.020595  0.000053      13        1   1.590508
14    -0.010904 -0.003039      14        1   1.353661
11    -0.008538  0.003624      15        1   1.240409
13    -0.022990 -0.036053      16        1   0.870216
15    -0.043826 -0.058110      17        1   0.502234
19    -0.007000  0.002314      18        1   0.095841
16    -0.037031 -0.065228      19        1   0.052728
17    -0.089036  0.083678      20        1   0.000556, topic_info=     Category       Freq        Term      Total  loglift  logprob
6629  Default   8.000000   tokarczuk   8.000000  30.0000  30.0000
652   Default   8.000000       power   8.000000  29.0000  29.0000
124   Default  43.000000         say  43.000000  28.0000  28.0000
2965  Default  18.000000        noon  18.000000  27.0000  27.0000
8143  Default   6.000000    guardant   6.000000  26.0000  26.0000
...       ...        ...         ...        ...      ...      ...
2769  Topic20   0.000165     halleck   7.537514   1.3700  -7.1861
964   Topic20   0.000157  university   6.562334   1.4616  -7.2330
2122  Topic20   0.000158       royal   6.701037   1.4416  -7.2321
2474  Topic20   0.000149   augustine   5.000610   1.6806  -7.2859
6629  Topic20   0.000153   tokarczuk   8.778685   1.1442  -7.2595

[1066 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7429      3  0.446104  abdominal
7429      5  0.223052  abdominal
761       1  0.749641       abiy
761       2  0.107092       abiy
6980      7  0.339961    abramov
...     ...       ...        ...
2241      1  0.263037        郑焕松
2241      2  0.131518        郑焕松
2241      4  0.131518        郑焕松
2241      5  0.131518        郑焕松
2241      8  0.131518        郑焕松

[2479 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 6, 7, 8, 9, 11, 10, 5, 13, 19, 15, 12, 14, 16, 20, 17, 18])